In [1]:
import json
import pandas as pd
from tqdm import tqdm

In [2]:
from collections import defaultdict
import random
import json
import unidecode
from indicnlp.normalize import indic_normalize
from indicnlp.transliterate import unicode_transliterate
from indicnlp.tokenize import indic_tokenize
from sacremoses import MosesPunctNormalizer
from sacremoses import MosesTokenizer
import urduhack
import os

2022-08-27 14:21:33.067187: W tensorflow/stream_executor/platform/default/dso_loader.cc:59] Could not load dynamic library 'libcudart.so.10.1'; dlerror: libcudart.so.10.1: cannot open shared object file: No such file or directory
2022-08-27 14:21:33.067222: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
/home2/user/miniconda3/lib/python3.7/site-packages/tensorflow_addons/utils/ensure_tf_install.py:67: UserWarning: Tensorflow Addons supports using Python ops for all Tensorflow versions above or equal to 2.6.0 and strictly below 2.9.0 (nightly versions are not supported). 
 The versions of TensorFlow you are currently using is 2.3.0 and is not supported. 
Some things might work, some things might not.
If you were to encounter a bug, do not file an issue.
If you want to make sure you're using a tested and supported configuration, either change the TensorFlow version or the TensorFlow Addons's version. 
You 

In [3]:
languages_map = {
    'en': {"label": "English", 'id': 0},
    'hi': {"label": "Hindi", 'id': 1},
    'te': {"label": "Telugu", 'id': 2}, 
    'bn': {"label": "Bengali", 'id': 3},
    'pa': {"label": "Punjabi", 'id': 4},
    'ur': {"label": "Urdu", 'id': 5}, 
    'or': {"label": "Odia", 'id': 6}, 
    'as': {"label": "Assamese", 'id': 7},
    'gu': {"label": "Gujarati", 'id': 8},
    'mr': {"label": "Marathi", 'id': 9},
    'kn': {"label": "Kannada", 'id': 10},
    'ta': {"label": "Tamil", 'id': 11},
    'ml': {"label": "Malayalam", 'id': 12} 
}

In [4]:
def get_language_normalizer():
    lang_normalizer = {}
    for k in languages_map:
        if k=='ur':
            lang_normalizer[k] = urduhack.normalization
        elif k=='en':
            lang_normalizer[k] = MosesPunctNormalizer()
        else:
            normfactory = indic_normalize.IndicNormalizerFactory()
            lang_normalizer[k] = normfactory.get_normalizer(k)
    return lang_normalizer

In [5]:
lang_normalizer = get_language_normalizer()

In [6]:
def get_text_in_unified_script(text, normalizer, lang):
    return unicode_transliterate.UnicodeIndicTransliterator.transliterate(
                " ".join(indic_tokenize.trivial_tokenize(normalizer.normalize(text.strip()), lang)),lang,"hi",).replace(" ् ", "्")

In [7]:
def run(fp_read, fp_write):
    with open(fp_read, 'r') as fr, open(fp_write, 'a') as fw:
        for line in fr:
            data = json.loads(line)

            triples = []
            subject = data['entity_name']
            triples.append([' <H> '+subject])
            for triple in data['facts']:
                predicate = triple[0]
                object = triple[1]
                triples.append([' <R> '+predicate, ' <T> '+object])

                for quals in triple[2]:
                    qual_subject = object
                    qual_predicate = quals[0]
                    qual_object = quals[1]
                    triples.append([' <QR> '+qual_predicate, ' <QT> '+qual_object])
            
            target_text = data['sentence']

            fw.write(json.dumps(triples) + '\t' + data['lang'] + '\t' + target_text.strip(",\/ \" \\") + '\t' + get_text_in_unified_script(target_text.strip(",\/ \" \\"), normalizer=lang_normalizer[data['lang']], lang=data['lang']) + '\n')


In [7]:
def run(fp_read, fp_write):
    with open(fp_read, 'r') as fr, open(fp_write, 'a') as fw:
        for line in fr:
            data = json.loads(line)

            triples = []
            subject = data['entity_name']
            # triples.append([' <S> '+subject])
            for triple in data['facts']:
                predicate = triple[0]
                object = triple[1]
                triples.append([' '+subject, ' '+predicate, ' '+object])

                for quals in triple[2]:
                    qual_subject = object
                    qual_predicate = quals[0]
                    qual_object = quals[1]
                    triples.append([' '+qual_subject, ' '+qual_predicate, ' '+qual_object])
            
            target_text = data['sentence']

            fw.write(json.dumps(triples) + '\t' + data['lang'] + '\t' + target_text.strip(",\/ \" \\") + '\t' + get_text_in_unified_script(target_text.strip(",\/ \" \\"), normalizer=lang_normalizer[data['lang']], lang=data['lang']) + '\n')


In [7]:
def run(fp_read, fp_write):
    with open(fp_read, 'r') as fr, open(fp_write, 'a') as fw:
        for line in fr:
            data = json.loads(line)

            triples = []
            subject = data['entity_name']
            # triples.append([' <S> '+subject])
            for triple in data['facts']:
                predicate = triple[0]
                object = triple[1]
                triples.append([' S| '+subject, ' P| '+predicate, ' O| '+object])

                for quals in triple[2]:
                    qual_subject = object
                    qual_predicate = quals[0]
                    qual_object = quals[1]
                    triples.append([' S| '+qual_subject, ' P| '+qual_predicate, ' O| '+qual_object])
            
            target_text = data['sentence']
            suffix = data['section']
            fw.write(json.dumps(triples, ensure_ascii=False) + '\t' + data['lang'] + '\t' + target_text.strip(",\/ \" \\") + '\t' + get_text_in_unified_script(target_text.strip(",\/ \" \\"), normalizer=lang_normalizer[data['lang']], lang=data['lang']) + '\n')


In [6]:
def run(fp_read, fp_write):
    with open(fp_read, 'r') as fr, open(fp_write, 'a') as fw:
        for line in fr:
            data = json.loads(line)

            triples = []
            subject = data['xlit_entity_name']
            # triples.append([' <S> '+subject])
            for triple in data['xlit_facts']:
                predicate = triple[0]
                object = triple[1]
                triples.append([' S| '+subject, ' P| '+predicate, ' O| '+object])

                for quals in triple[2]:
                    qual_subject = object
                    qual_predicate = quals[0]
                    qual_object = quals[1]
                    triples.append([' S| '+qual_subject, ' P| '+qual_predicate, ' O| '+qual_object])
            
            target_text = data['sentence']

            fw.write(json.dumps(triples, ensure_ascii=False) + '\t' + data['lang'] + '\t' + target_text.strip(",\/ \" \\") + '\t' + get_text_in_unified_script(target_text.strip(",\/ \" \\"), normalizer=lang_normalizer[data['lang']], lang=data['lang']) + '\n')


In [7]:
def run(fp_read, fp_write):
    with open(fp_read, 'r') as fr, open(fp_write, 'a') as fw:
        for line in fr:
            data = json.loads(line)

            triples = []
            subject = data['entity_name']
            # triples.append([' <S> '+subject])
            for triple in data['facts']:
                predicate = triple[0]
                object = triple[1]
                triples.append([' <H> '+subject, ' <R> '+predicate, ' <T> '+object])

                for quals in triple[2]:
                    qual_subject = object
                    qual_predicate = quals[0]
                    qual_object = quals[1]
                    triples.append([' <H> '+qual_subject, ' <R> '+qual_predicate, ' <T> '+qual_object])
            
            target_text = data['sentence']

            fw.write(json.dumps(triples) + '\t' + data['lang'] + '\t' + target_text.strip(",\/ \" \\") + '\t' + get_text_in_unified_script(target_text.strip(",\/ \" \\"), normalizer=lang_normalizer[data['lang']], lang=data['lang']) + '\n')


In [7]:
# for ordered facts and HRT only
def run(fp_read, fp_write):
    with open(fp_read, 'r') as fr, open(fp_write, 'a') as fw:
        for line in fr:
            data = json.loads(line)

            triples = []
            subject = data['entity_name']
            ordered_facts = [data['facts'][i] for i in data['role_spec_order']]
            for triple in ordered_facts:
                predicate = triple[0]
                object = triple[1]
                triples.append([' <H> '+subject, ' <R> '+predicate, ' <T> '+object])

                for quals in triple[2]:
                    qual_subject = object
                    qual_predicate = quals[0]
                    qual_object = quals[1]
                    triples.append([' <H> '+qual_subject, ' <R> '+qual_predicate, ' <T> '+qual_object])
            
            target_text = data['sentence']

            fw.write(json.dumps(triples) + '\t' + data['lang'] + '\t' + target_text.strip(",\/ \" \\") + '\t' + get_text_in_unified_script(target_text.strip(",\/ \" \\"), normalizer=lang_normalizer[data['lang']], lang=data['lang']) + '\n')


In [7]:
def run(fp_read, fp_write):
    with open(fp_read, 'r') as fr, open(fp_write, 'a') as fw:
        for line in fr:
            data = json.loads(line)

            triples = []
            subject = data['xlit_entity_name']
            # triples.append([' <S> '+subject])
            for triple in data['xlit_facts']:
                predicate = triple[0]
                object = triple[1]
                triples.append([' <S> '+subject, ' <P> '+predicate, ' <O> '+object])

                for quals in triple[2]:
                    qual_subject = object
                    qual_predicate = quals[0]
                    qual_object = quals[1]
                    triples.append([' <S> '+qual_subject, ' <P> '+qual_predicate, ' <O> '+qual_object])
            
            target_text = data['sentence']

            fw.write(json.dumps(triples, ensure_ascii=False) + '\t' + data['lang'] + '\t' + target_text.strip(",\/ \" \\") + '\t' + get_text_in_unified_script(target_text.strip(",\/ \" \\"), normalizer=lang_normalizer[data['lang']], lang=data['lang']) + '\n')


In [ ]:
for mode in ['train', 'val', 'test']:
    fp_write = f'role_ordered_xalign_HRT_only/{mode}.csv'
    
    with open(fp_write, 'w') as fw:
        fw.write('input' + '\t' + 'lang' + '\t' + 'original_script' + '\t' + 'target' + '\n')
    
    for lang in ['bn', 'en', 'gu', 'hi', 'kn', 'mr', 'ta', 'te', 'ml', 'pa', 'or', 'as']:
        fp_read = f'/home2/user/stuff/copernicus/xalign_data/datasets_v2.7/{lang}/{mode}.jsonl'
        run(fp_read, fp_write)